# Model: Skulls v0

---
# Load data & libraries

In [1]:
# TOP VERSION
MODEL_NAME = "skulls"
VERSION = 0

In [2]:
# import dependencies
import pandas as pd
from lightgbm import LGBMRegressor
import gc
import json
from pathlib import Path
from numerapi import NumerAPI
from utils.utils import (
    save_model,
    load_model,
    ERA_COL,
    DATA_TYPE_COL,
    TARGET_COL,
)
from utils.api_keys import PUBLIC_ID, SECRET_KEY

In [3]:
# instantiate numerai api
napi = NumerAPI(public_id=PUBLIC_ID, secret_key=SECRET_KEY)
current_round = napi.get_current_round()
print(f"Current round #: {current_round}")

Current round #: 316


In [4]:
# read the feature metadata and get the "medium" feature set
with open("data/top_bottom_features.json", "r") as f:
    top_bottom_features = json.load(f)
features = top_bottom_features["top_features"] + top_bottom_features["bottom_features"]

read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

training_data = pd.read_parquet('data/train.parquet', columns=read_columns)
validation_data = pd.read_parquet('data/validation.parquet', columns=read_columns)
live_data = pd.read_parquet(f'data/live_{current_round}.parquet', columns=read_columns)

---
# Create model

In [5]:
# model name
model_name = f"dh_{MODEL_NAME}_v{VERSION}"
model_name

'dh_skulls_v0'

In [6]:
# check existing model, if not, train it
print(f"Checking for existing model: {model_name}...")
model = load_model(model_name)

if not model:
    print(f"Model not found, creating new one")
    params = {"n_estimators": 2000,
              "learning_rate": 0.01,
              "max_depth": 5,
              "num_leaves": 2 ** 5,
              "colsample_bytree": 0.1,
              "verbosity": 1}
    model = LGBMRegressor(**params)
    
    print(f"Training model: {model_name}...")
    model.fit(
        training_data.filter(like='feature_', axis='columns'),
        training_data[TARGET_COL]
    )
    
    print(f"Saving new model: {model_name}...")
    save_model(model, model_name)

Checking for existing model: dh_skulls_v0...


In [7]:
# clear memory of unused data
gc.collect()

0

---
# Create predictions

In [8]:
# check for nans
nans_per_col = live_data[live_data["data_type"] == "live"].isna().sum()

if nans_per_col.any():
    total_rows = len(live_data[live_data["data_type"] == "live"])
    print(f"Number of nans per column this week: {nans_per_col[nans_per_col > 0]}")
    print(f"out of {total_rows} total rows")
    print(f"filling nans with 0.5")
    live_data.loc[:, features] = live_data.loc[:, features].fillna(0.5)
else:
    print("No nans in the features this week!")

Number of nans per column this week: target_nomi_v4_20    5261
dtype: int64
out of 5261 total rows
filling nans with 0.5


In [9]:
# check feature name
model_expected_features = model.booster_.feature_name()

validation_data.loc[:, f"preds_{model_name}"] = model.predict(validation_data.loc[:, model_expected_features])
live_data.loc[:, f"preds_{model_name}"] = model.predict(live_data.loc[:, model_expected_features])

In [10]:
# clear memory of unused data
gc.collect()

24

---
# Save predictions

In [11]:
# rename best model to "prediction" and rank from 0 to 1 to meet upload requirements
model_to_submit = f"preds_{model_name}"

validation_data["prediction"] = validation_data[model_to_submit].rank(pct=True)
live_data["prediction"] = live_data[model_to_submit].rank(pct=True)
validation_data["prediction"].to_csv(f"predictions/{model_name}_val_preds.csv")
live_data["prediction"].to_csv(f"predictions/{model_name}_live_preds_{current_round}.csv")